# DApps with DAOs transaction history panel preparation

This script prepares a panel data set that contains daily usage data for dApps (including dApps are governed by a DAO)


Daily usage data is retrieved from the Google Big Query Ethereum public data set

## Import the data set

In [147]:
import pandas as pd
from itertools import product
import datetime as dt
import numpy as np


In [63]:
daily_dapp_usage = pd.read_csv(r"ethereum_dapps_dao_daily_usage.csv")
daily_dapp_usage.head()

,date_key,dapp_name,DAO_space_id,transactions,eoa,SUM_txn_value_GWEI,AVG_txn_value_GWEI
0,2022-05-22,Rare Land NFT,no_dao,15652,9031,0.000000e+00,0.000000
1,2022-01-31,BRAVE FRONTIER HEROES,no_dao,7625,6951,1.932924e+09,253498.188108
2,2018-02-17,RenVM,no_dao,3111,1878,0.000000e+00,0.000000
3,2021-03-23,Axie Infinity,no_dao,3434,733,2.200000e+09,640652.300524
4,2020-10-21,MakersPlace,no_dao,196,11,0.000000e+00,0.000000


## Expand panel to all days

In [140]:
# create an empty panel with all dates

# create a date range
date_range = pd.date_range(start='2020-01-11', end=daily_dapp_usage.date_key.max())

# create list of unique dapp names
dapps = daily_dapp_usage.dapp_name.unique()

# get permutation of dapp names and dates
data = list(product(date_range, dapps))

# convert list of tuples to data frame
empty_panel = pd.DataFrame.from_records(data, columns =['date_key', 'dapp_name'])

# convert timestamp to string
empty_panel["date_key"] = empty_panel.date_key.dt.strftime('%Y-%m-%d')

# merge both data frames
dapp_usage_panel = pd.merge(empty_panel, daily_dapp_usage,  how='left', left_on=['date_key','dapp_name'], right_on = ['date_key', 'dapp_name'])

# add dao names without NaNs
dapp_usage_panel = dapp_usage_panel.drop("DAO_space_id", axis = 1)

dapp_dao_names = daily_dapp_usage[["dapp_name","DAO_space_id"]].drop_duplicates()

dapp_usage_panel = pd.merge(dapp_usage_panel, dapp_dao_names,  how='left', left_on=['dapp_name'], right_on = ['dapp_name'])


## Add category of the dApp

The dapp data is scraped from DAPP radar

In [141]:
# import the scraped dapp info data 
dapps_info = pd.read_csv(r"ethereum_dapps.csv", index_col=0)
dapps_info[["name", "category"]].head()

,name,category
0,OpenSea,Marketplaces
1,Uniswap V2,Exchanges
2,Uniswap V3,Exchanges
3,MetaMask Swap,DeFi
4,Gem,Marketplaces


In [142]:
# merge the category column to the dapp usage panel
dapp_usage_panel = pd.merge(dapp_usage_panel, dapps_info[["name", "category"]],  how='left', left_on=['dapp_name'], right_on = ['name'])
dapp_usage_panel = dapp_usage_panel.drop("name", axis = 1)

dapp_usage_panel.head()

,date_key,dapp_name,transactions,eoa,SUM_txn_value_GWEI,AVG_txn_value_GWEI,DAO_space_id,category
0,2020-01-11,Rare Land NFT,NaN,NaN,NaN,NaN,no_dao,Collectibles
1,2020-01-11,BRAVE FRONTIER HEROES,52.0,51.0,0.000000e+00,0.000000e+00,no_dao,Games
2,2020-01-11,RenVM,82.0,46.0,0.000000e+00,0.000000e+00,no_dao,DeFi
3,2020-01-11,Axie Infinity,484.0,131.0,4.216000e+09,8.710744e+06,no_dao,Games
4,2020-01-11,MakersPlace,270.0,9.0,1.135000e+10,4.203704e+07,no_dao,Collectibles


## Add date of first proposal if the dApp has a DAO

In [117]:
# load sql extension and connect to the data base with magic commands
#%%capture
%load_ext sql

%sql sqlite:///snapshot_sqlite.db

In [ ]:
# get list of all spaces 
space_list = str(list(dapp_usage_panel.DAO_space_id.unique()))
print(space_list)

In [143]:
# find start date of proposal 
sql_result = %sql select space_id, min(start), min(end) from proposals WHERE space_id in ('no_dao', 'olympusdao.eth', 'dydxgov.eth', 'uniswap', 'cvx.eth', 'badgerdao.eth', 'masknetwork.eth', 'doodles.eth', 'ens.eth', 'tranche.eth', 'rallygov.eth', 'pooltogether.eth', 'origingov.eth', 'instadapp-gov.eth', 'baovotes.eth', 'ilvgov.eth', 'vote-perp.eth', 'vote.airswap.eth', 'frax.eth', 'ybaby.eth', 'fei.eth', 'lineardao.eth', 'gnosis.eth', 'index-coop.eth', 'community.nexusmutual.eth', 'cow.eth', 'nftx.eth', 'deversifi.eth', 'swerve', 'rarible.eth', 'aave.eth', 'snapshot.dcl.eth', 'aurafinance.eth', 'alchemixstakers.eth', 'fuse.eth', 'decentralgames.eth', 'sushigov.eth', 'cream-finance.eth', 'lido-snapshot.eth', 'piedao.eth', 'vsp.eth', 'pickle.eth', 'boringvote.eth', 'polymon.eth', 'usebraintrust.eth', 'synapseprotocol.eth', 'tokenlon.eth', 'meritcircle.eth', 'gro.xyz', 'saddlefinance.eth', 'truefigov.eth', 'thalesgov.eth', 'zombietoadzdao.eth', 'bancornetwork.eth', 'dodobird.eth', 'notional.eth', 'finnexus', 'draculasucks.eth', 'theopendao.eth', 'dinobabies.eth', 'uniclynft.eth', 'yokaihouse.eth', 'staking.idlefinance.eth', 'balancer.eth', 'mutantsdao.eth', 'barnbridge.eth', 'expansiondao.eth', 'theheaddao.eth', 'monsterbuds.eth', 'svs.eth', 'halloweenbearsdao.eth', 'rot', 'cultdaoart.eth', 'goopsnapshot.eth', 'ancientshrooms.eth', 'octofi', 'togetherstrong.eth', 'elyfi-bsc.eth', 'gov.dhedge.eth', 'dforcenet.eth', 'ethlizards.eth', 'zombiesdao.eth', 'unfi.eth', 'skullsdao.eth', 'apy.eth', 'thehashes.eth', 'yam.eth', 'popcorn-snapshot.eth', 'stackedtoadzdao.eth', 'cryptohoots.eth', 'mimo.eth', 'inversefinance.eth', 'giv.eth', 'scaleswap.eth', 'idlefinance.eth', 'tracer.eth', 'epns.eth', 'fortube-eth', 'mstabledao.eth', 'fraktal.eth') GROUP BY 1
first_proposals = sql_result.DataFrame()

first_proposals["start_first_prop"] = pd.to_datetime(first_proposals['min(start)'],unit='s').dt.strftime('%Y-%m-%d')
first_proposals["end_first_prop"] = pd.to_datetime(first_proposals['min(end)'],unit='s').dt.strftime('%Y-%m-%d')

first_proposals = first_proposals.drop("min(start)", axis = 1)
first_proposals = first_proposals.drop("min(end)", axis = 1)

first_proposals

 * sqlite:///snapshot_sqlite.db
   sqlite:///snapshot_sqlite_nov2022.db
Done.


,space_id,start_first_prop,end_first_prop
0,aave.eth,2021-06-21,2021-06-24
1,alchemixstakers.eth,2021-02-28,2021-03-01
2,ancientshrooms.eth,2021-12-13,2021-12-15
3,apy.eth,2021-09-24,2021-10-08
4,aurafinance.eth,2022-06-16,2022-06-19
...,...,...,...
94,yam.eth,2020-08-23,2020-08-26
95,ybaby.eth,2021-04-08,2021-04-13
96,yokaihouse.eth,2022-03-14,2022-03-19
97,zombiesdao.eth,2021-10-08,2021-10-08


In [ ]:
# merge start and end date of first proposal to dapp usage panel
dapp_usage_panel = pd.merge(dapp_usage_panel, first_proposals,  how='left', left_on=['DAO_space_id'], right_on = ['space_id'])

dapp_usage_panel = dapp_usage_panel.drop("space_id", axis = 1)

#dapp_usage_panel.head()

In [149]:
# creat a dummy variables that indicates of a dapp has a DAO
dapp_usage_panel['has_dao'] = np.where(dapp_usage_panel['DAO_space_id'] !='no_dao', 1, 0)

In [156]:
# convert date strings to date time objects
dapp_usage_panel['date_key'] = pd.to_datetime(dapp_usage_panel['date_key'], format='%Y-%m-%d')
dapp_usage_panel['start_first_prop'] = pd.to_datetime(dapp_usage_panel['start_first_prop'], format='%Y-%m-%d')
dapp_usage_panel['end_first_prop'] = pd.to_datetime(dapp_usage_panel['end_first_prop'], format='%Y-%m-%d')

dapp_usage_panel['intro_dao_start_prop'] = np.where(dapp_usage_panel['date_key'] >= dapp_usage_panel['start_first_prop'], 1, 0)
dapp_usage_panel['intro_dao_pro_prop'] = np.where(dapp_usage_panel['date_key'] >= dapp_usage_panel['end_first_prop'], 1, 0)

## Export data set

In [161]:
dapp_usage_panel.head()

,date_key,dapp_name,transactions,eoa,SUM_txn_value_GWEI,AVG_txn_value_GWEI,DAO_space_id,category,start_first_prop,end_first_prop,has_dao,intro_dao_start_prop,intro_dao_pro_prop
0,2020-01-11,Rare Land NFT,NaN,NaN,NaN,NaN,no_dao,Collectibles,NaT,NaT,0,0,0
1,2020-01-11,BRAVE FRONTIER HEROES,52.0,51.0,0.000000e+00,0.000000e+00,no_dao,Games,NaT,NaT,0,0,0
2,2020-01-11,RenVM,82.0,46.0,0.000000e+00,0.000000e+00,no_dao,DeFi,NaT,NaT,0,0,0
3,2020-01-11,Axie Infinity,484.0,131.0,4.216000e+09,8.710744e+06,no_dao,Games,NaT,NaT,0,0,0
4,2020-01-11,MakersPlace,270.0,9.0,1.135000e+10,4.203704e+07,no_dao,Collectibles,NaT,NaT,0,0,0


In [160]:
dapp_usage_panel.to_csv('dapp_dao_usage_panel.csv', sep=";")